<font color=black size=4 face=雅黑>
    
1. 已有包含职位名称和岗位描述的数据集
2. 对岗位描述进行分词,得到关键词
3. 关键词和对应的职位名称相关
4. 现在新增一条包含岗位描述的记录
5. 希望可以从新增的岗位描述中,根据已有的数据,提取出和潜在职位相关的关键词
6. 将为新岗位描述提取出的关键词,添加到原数据集对应职位的“关键词”列中

可以通过以下步骤实现:

1. 使用已有数据训练一个文本分类模型,输入是岗位描述,输出是职位名称。
2. 对新增的岗位描述,使用训练好的模型进行预测,得到潜在的职位名称。
3. 根据模型输出的职位名称,找到与其对应的关键词。
4. 从新增岗位描述中提取出这些关键词作为结果。
5. 主要利用机器学习模型学习岗位描述和职位名称之间的关联,然后反向推理出描述对应的关键词。
6. 找到这个职位名称在数据集中对应的行，去重，将新关键词添加到该行的关键词字段后面

</font>

In [37]:
import pandas as pd

# 文件路径
file_path = r"C:\Users\Xinru.Lee\jobs1.csv"

# 读取文件
data = pd.read_csv(file_path, encoding='utf-8')
print(data, '\n')
 
#查看前3列数据，默认是5行
print(data.head(3), '\n')
 
#查看数据的形状，返回（行数,列数）
print(data.shape, '\n')
 
#查看列名列表
print(data.columns, '\n')
 
#查看索引列
print(data.index, '\n')

                岗位名称  类别         职位名称                  所属地       专业学科   学历要求  \
0    002.建筑材料试验员（济南）  正式        建筑工程师  济南市 青岛市 临沂市 济宁市 泰安市    材料科学与工程  本科及以上   
1                NaN  正式        建筑工程师  济南市 青岛市 临沂市 济宁市 泰安市  无机非金属材料工程  本科及以上   
2    007主体钢结构检测员（济南）  正式         建筑工程      济南市 青岛市 济宁市 临沂市        建筑类  本科及以上   
3                NaN  正式         建筑工程      济南市 青岛市 济宁市 临沂市       工程管理  本科及以上   
4                NaN  正式         建筑工程      济南市 青岛市 济宁市 临沂市       土木工程  本科及以上   
..               ...  ..          ...                  ...        ...    ...   
876            化妆品销售  正式        代理商销售              山东省 济南市       电子商务     不限   
877       店长（全国就近派遣）  正式  门店经理/店长店长助理                   全国       不限专业     不限   
878         java开发实习  实习    Java开发工程师                   全国       不限专业     不限   
879           UI设计实习  实习       视觉设计经理                   全国       不限专业     不限   
880        web前端开发实习  实习      Web前端开发                   全国       不限专业     不限   

                                       

In [38]:
import numpy as np

print(np.array(data[["岗位描述"]]).tolist())

[['1、材料、建筑、土木、电工、高分子材料、消防相关专业;2、吃苦耐劳，责任心强，有驾照者优先考虑，可接受实习生或应届毕业生；3、能够熟练使用办公软件，有良好的沟通能力、组织协调能力和团队协作精神；4、有检验检测行业相关工作经验者优先。'], ['1、材料、建筑、土木、电工、高分子材料、消防相关专业;2、吃苦耐劳，责任心强，有驾照者优先考虑，可接受实习生或应届毕业生；3、能够熟练使用办公软件，有良好的沟通能力、组织协调能力和团队协作精神；4、有检验检测行业相关工作经验者优先。'], ['1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强的学习能力及执行力；3、有驾驶经验者优先考虑。'], ['1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强的学习能力及执行力；3、有驾驶经验者优先考虑。'], ['1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强的学习能力及执行力；3、有驾驶经验者优先考虑。'], ['1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强的学习能力及执行力；3、有驾驶经验者优先考虑。'], ['1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强的学习能力及执行力；3、有驾驶经验者优先考虑。'], ['1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强的学习能力及执行力；3、有驾驶经验者优先考虑。'], ['任职要求：1、优秀应届毕业生及实习生，本科及以上学历，材料、建筑、土木、高分子材料等相关专业优先考虑；2、工作积极主动，乐于沟通，有较强的学习能力及适应能力；3、综合素质较好，具备良好的组织、协调能力，有一定抗压能力及管理潜质；4、有意向从事检测行业。6、档案关系保障：可接收人事、党员档案关系（公司设有党支部），可评职称；7、岗位晋升：完善的晋升制度，技术、管理、市场三通道职业发展路线，为每一个有梦想、求发展、敢于拼搏的人提供公平合理的晋升平台，我们做的不只是一份工作，更是未来的事业！'], ['任职要求：1、优秀应届毕业生及实习生，本科及以上学历，材料、建筑、土木、高分子材料等相关专业优先考虑；2

# jieba分词

In [39]:
import jieba
import re

# 加载停用词表
stop_words = open('hit_stopwords.txt', 'r', encoding='utf-8').readlines() 
stop_words = [w.strip() for w in stop_words]

def preprocess(text):
    # 分词
    words = jieba.lcut(text.encode('utf-8'))
    # words = jieba.cut(text, cut_all=True)
    
    # 去停用词
    words = [w for w in words if w not in stop_words]
    
    # 去标点符号
    words = [re.sub(r'[^\w\s]', '', w) for w in words]
    
    # 去空格
    words = [w.strip() for w in words]
    
    # 去空行
    words = [w for w in words if w]
    
    return words

text = str(np.array(data[["岗位描述"]]).tolist())

words = preprocess(text)

print(words)

['1', '材料', '建筑', '土木', '电工', '高分子', '材料', '消防', '相关', '专业', '2', '吃苦耐劳', '责任心', '强', '驾照', '优先', '考虑', '接受', '实习生', '应届', '毕业生', '3', '能够', '熟练', '使用', '办公', '软件', '良好', '沟通', '能力', '组织协调', '能力', '团队', '协作', '精神', '4', '检验', '检测', '行业', '相关', '工作', '经验', '优先', '1', '材料', '建筑', '土木', '电工', '高分子', '材料', '消防', '相关', '专业', '2', '吃苦耐劳', '责任心', '强', '驾照', '优先', '考虑', '接受', '实习生', '应届', '毕业生', '3', '能够', '熟练', '使用', '办公', '软件', '良好', '沟通', '能力', '组织协调', '能力', '团队', '协作', '精神', '4', '检验', '检测', '行业', '相关', '工作', '经验', '优先', '1', '建工', '土木', '材料', '机械', '相关', '专业', '接受', '实习生', '应届', '毕业生', '2', '出差', '吃苦耐劳', '强', '学习', '能力', '执行力', '3', '驾驶', '经验', '优先', '考虑', '1', '建工', '土木', '材料', '机械', '相关', '专业', '接受', '实习生', '应届', '毕业生', '2', '出差', '吃苦耐劳', '强', '学习', '能力', '执行力', '3', '驾驶', '经验', '优先', '考虑', '1', '建工', '土木', '材料', '机械', '相关', '专业', '接受', '实习生', '应届', '毕业生', '2', '出差', '吃苦耐劳', '强', '学习', '能力', '执行力', '3', '驾驶', '经验', '优先', '考虑', '1', '建工', '土木', '材料', '机械', '相关', '专业', '接受', '实习生', '应届', 

# 加载哈工大LTP4.0模型

In [40]:
import torch
from ltp import LTP
ltp = LTP("LTP/base2") # 默认加载 LTP/Small 模型
# ltp = LTP(path = "LTP/base|LTP/small|LTP/tiny")

# test
output = ltp.pipeline(["他叫汤姆去拿外衣。"], tasks=["cws"])
# 使用字典格式作为返回结果
print(output.cws)
# 使用元组格式作为返回结果
cws = ltp.pipeline(["他叫汤姆去拿外衣。"], tasks=["cws"]).to_tuple()  # to tuple 可以自动转换为元组格式
print(cws)

Loading weights from local directory
[['他', '叫', '汤姆', '去', '拿', '外衣', '。']]
([['他', '叫', '汤姆', '去', '拿', '外衣', '。']],)


# LTP分词

In [41]:
def preprocess_ltp(text):
    # 分词
    words_ltp = ltp.pipeline([text], tasks=["cws"])
    # words = jieba.cut(text, cut_all=True)
    
    # 去停用词
    words_ltp = [w for w in words_ltp if w not in stop_words]
    
    # 去标点符号
    words_ltp = [re.sub(r'[^\w\s]', '', w) for w in words_ltp]
    
    # 去空格
    words_ltp = [w.strip() for w in words_ltp]
    
    # 去空行
    words_ltp = [w for w in words_ltp if w]
    
    return words_ltp

words_ltp = preprocess_ltp(text)

print(words_ltp)

['cws']


In [42]:
df = data
df

,岗位名称,类别,职位名称,所属地,专业学科,学历要求,岗位描述
0,002.建筑材料试验员（济南）,正式,建筑工程师,济南市 青岛市 临沂市 济宁市 泰安市,材料科学与工程,本科及以上,1、材料、建筑、土木、电工、高分子材料、消防相关专业;2、吃苦耐劳，责任心强，有驾照者优先考...
1,NaN,正式,建筑工程师,济南市 青岛市 临沂市 济宁市 泰安市,无机非金属材料工程,本科及以上,1、材料、建筑、土木、电工、高分子材料、消防相关专业;2、吃苦耐劳，责任心强，有驾照者优先考...
2,007主体钢结构检测员（济南）,正式,建筑工程,济南市 青岛市 济宁市 临沂市,建筑类,本科及以上,1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强...
3,NaN,正式,建筑工程,济南市 青岛市 济宁市 临沂市,工程管理,本科及以上,1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强...
4,NaN,正式,建筑工程,济南市 青岛市 济宁市 临沂市,土木工程,本科及以上,1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强...
...,...,...,...,...,...,...,...
876,化妆品销售,正式,代理商销售,山东省 济南市,电子商务,不限,任职要求：1、相关专业大专以上学历；2、校园网红、创业经历或者校园代理经验优先考虑；3、热爱...
877,店长（全国就近派遣）,正式,门店经理/店长店长助理,全国,不限专业,不限,〖我们需要这样的你〗1.年龄 ：22-35周岁（有行业经验者，年龄可放宽）2.良好的沟通、领...
878,java开发实习,实习,Java开发工程师,全国,不限专业,不限,了解HTML /XML、CSS、 JSP、 Java语言;
879,UI设计实习,实习,视觉设计经理,全国,不限专业,不限,1、美术类，计算机类相关专业优先;2、熟悉Photoshop、lustrator、 Flas...


In [43]:
# 增加'关键词'列,并逐行加入分词结果  
df['关键词'] = ''

for index, row in df.iterrows():
    text = row['岗位描述']
    words = preprocess(text)
    df.at[index, '关键词'] = ",".join(words)

df

,岗位名称,类别,职位名称,所属地,专业学科,学历要求,岗位描述,关键词
0,002.建筑材料试验员（济南）,正式,建筑工程师,济南市 青岛市 临沂市 济宁市 泰安市,材料科学与工程,本科及以上,1、材料、建筑、土木、电工、高分子材料、消防相关专业;2、吃苦耐劳，责任心强，有驾照者优先考...,"1,材料,建筑,土木,电工,高分子,材料,消防,相关,专业,2,吃苦耐劳,责任心,强,驾照,..."
1,NaN,正式,建筑工程师,济南市 青岛市 临沂市 济宁市 泰安市,无机非金属材料工程,本科及以上,1、材料、建筑、土木、电工、高分子材料、消防相关专业;2、吃苦耐劳，责任心强，有驾照者优先考...,"1,材料,建筑,土木,电工,高分子,材料,消防,相关,专业,2,吃苦耐劳,责任心,强,驾照,..."
2,007主体钢结构检测员（济南）,正式,建筑工程,济南市 青岛市 济宁市 临沂市,建筑类,本科及以上,1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强...,"1,建工,土木,材料,机械,相关,专业,接受,实习生,应届,毕业生,2,出差,吃苦耐劳,强,..."
3,NaN,正式,建筑工程,济南市 青岛市 济宁市 临沂市,工程管理,本科及以上,1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强...,"1,建工,土木,材料,机械,相关,专业,接受,实习生,应届,毕业生,2,出差,吃苦耐劳,强,..."
4,NaN,正式,建筑工程,济南市 青岛市 济宁市 临沂市,土木工程,本科及以上,1、建工、土木、材料、机械等相关专业，可接受实习生或应届毕业生；2、可出差，吃苦耐劳，有较强...,"1,建工,土木,材料,机械,相关,专业,接受,实习生,应届,毕业生,2,出差,吃苦耐劳,强,..."
...,...,...,...,...,...,...,...,...
876,化妆品销售,正式,代理商销售,山东省 济南市,电子商务,不限,任职要求：1、相关专业大专以上学历；2、校园网红、创业经历或者校园代理经验优先考虑；3、热爱...,"任职,要求,1,相关,专业,大专,以上学历,2,校园网,红,创业,经历,校园,代理,经验,优..."
877,店长（全国就近派遣）,正式,门店经理/店长店长助理,全国,不限专业,不限,〖我们需要这样的你〗1.年龄 ：22-35周岁（有行业经验者，年龄可放宽）2.良好的沟通、领...,"需要,1,年龄,22,35,周岁,行业,经验,年龄,放宽,2,良好,沟通,领导,协调,能力,..."
878,java开发实习,实习,Java开发工程师,全国,不限专业,不限,了解HTML /XML、CSS、 JSP、 Java语言;,"了解,HTML,XML,CSS,JSP,Java,语言"
879,UI设计实习,实习,视觉设计经理,全国,不限专业,不限,1、美术类，计算机类相关专业优先;2、熟悉Photoshop、lustrator、 Flas...,"1,美术,类,计算机,类,相关,专业,优先,2,熟悉,Photoshop,lustrator..."


In [44]:
# 将增加了新列的DataFrame写入csv文件

# df.to_csv('job_keywords.csv', encoding='utf-8', index=False)

# df

# 模型预测职位

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# 文本特征提取
tfidf = TfidfVectorizer()

# 多项式贝叶斯分类器
nb = MultinomialNB()

# 构建管道
clf = Pipeline([('tfidf', tfidf), ('nb', nb)])

# 训练模型
clf.fit(df['岗位描述'], df['职位名称'])

# 新增岗位描述
new_desc = '1、教育程度：本科及以上学历，应往届相关专业毕业生均可；非相关专业功底优秀者也可；2、经验：熟悉山东地区教材内容和教育特点，有教学经验者更佳；3、技能：良好的学科功底，思路清晰，具备良好的亲和力以及沟通能力，热爱教育行业；' 

# 模型预测职位  
predicted_post = clf.predict([new_desc])

print('新岗位描述:', new_desc, '\n')
print('预测职位:', predicted_post[0]) 

新岗位描述: 1、教育程度：本科及以上学历，应往届相关专业毕业生均可；非相关专业功底优秀者也可；2、经验：熟悉山东地区教材内容和教育特点，有教学经验者更佳；3、技能：良好的学科功底，思路清晰，具备良好的亲和力以及沟通能力，热爱教育行业； 

预测职位: 中学教师


# 获取关键词

In [54]:
# 获取预测职位对应的旧关键词
old_keywords = df[df['职位名称']==predicted_post[0]]['关键词'].values[0]

# 获取预测职位对应的新关键词（即对新岗位描述分词）
# new_desc_keywords = preprocess(new_desc)
# new_desc_keywords = ltp.pipeline([new_desc], tasks=["cws"])
new_desc_keywords = preprocess_ltp(new_desc)
# new_desc_keywords = str(new_desc_keywords)

print('提取旧关键词:', old_keywords, '\n')
print('新岗位描述的分词结果', new_desc_keywords)

提取旧关键词: 1,教育,程度,本科,以上学历,应,往届,相关,专业,毕业生,均,非,相关,专业,功底,优秀者,2,经验,熟悉,山东地区,教材内容,教育,特点,教学,经验,更佳,3,技能,良好,学科,功底,思路清晰,具备,良好,亲和力,沟通,能力,热爱,教育,行业 

新岗位描述的分词结果 ['cws']


In [50]:
# 去停用词
# new_desc_keywords = [w for w in new_desc_keywords if w not in stop_words]
    
# 去标点符号
# new_desc_keywords = [re.sub(r'[^\w\s]', '', w) for w in new_desc_keywords]

# 去空格
# new_desc_keywords = [w.strip() for w in new_desc_keywords]
    
# 去空行
# new_desc_keywords = [w for w in new_desc_keywords if w]

print(new_desc_keywords.cws)

AttributeError: 'str' object has no attribute 'cws'

# 对预测职位对应的关键词进行去重

In [51]:
old_desc_keywords = preprocess(old_keywords)
old_desc_keywords

['1',
 '教育',
 '程度',
 '本科',
 '以上学历',
 '应',
 '往届',
 '相关',
 '专业',
 '毕业生',
 '均',
 '非',
 '相关',
 '专业',
 '功底',
 '优秀者',
 '2',
 '经验',
 '熟悉',
 '山东地区',
 '教材内容',
 '教育',
 '特点',
 '教学',
 '经验',
 '更佳',
 '3',
 '技能',
 '良好',
 '学科',
 '功底',
 '思路清晰',
 '具备',
 '良好',
 '亲和力',
 '沟通',
 '能力',
 '热爱',
 '教育',
 '行业']

# 根据索引找到对应的旧关键词，去重

In [52]:
new_desc_keywords = str(new_desc_keywords)
old_desc_keywords = str(old_desc_keywords)
print(new_desc_keywords, '\n')
print(old_desc_keywords)

['__annotations__', '__class__', '__class_getitem__', '__contains__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'cws', 'dep', 'fromkeys', 'get', 'items', 'keys', 'move_to_end', 'ner', 'pop', 'popitem', 'pos', 'sdp', 'sdpg', 'setdefault', 'srl', 'to_tuple', 'update', 'values'] 

['1', '教育', '程度', '本科', '以上学历', '应', '往届', '相关', '专业', '毕业生', '均', '非', '相关', '专业', '功底', '优秀者', '2', '经验', '熟悉', '山东地区', '教材内容', '教育', '特点', '教学', '经验', '更佳', '3', '技能', '良好', '学科', '功底', '思路清晰', '具备', '良好', '亲和力', '沟通', '能力', '热爱', '教育', '行业']


In [53]:
# 将新旧关键词合并
all_keywords = old_desc_keywords.split(',') + new_desc_keywords.split(',')  

print(all_keywords, '\n')

# 去重
all_keywords = set(all_keywords)

print(all_keywords)

["['1'", " '教育'", " '程度'", " '本科'", " '以上学历'", " '应'", " '往届'", " '相关'", " '专业'", " '毕业生'", " '均'", " '非'", " '相关'", " '专业'", " '功底'", " '优秀者'", " '2'", " '经验'", " '熟悉'", " '山东地区'", " '教材内容'", " '教育'", " '特点'", " '教学'", " '经验'", " '更佳'", " '3'", " '技能'", " '良好'", " '学科'", " '功底'", " '思路清晰'", " '具备'", " '良好'", " '亲和力'", " '沟通'", " '能力'", " '热爱'", " '教育'", " '行业']", "['__annotations__'", " '__class__'", " '__class_getitem__'", " '__contains__'", " '__dataclass_fields__'", " '__dataclass_params__'", " '__delattr__'", " '__delitem__'", " '__dict__'", " '__dir__'", " '__doc__'", " '__eq__'", " '__format__'", " '__ge__'", " '__getattribute__'", " '__getitem__'", " '__gt__'", " '__hash__'", " '__init__'", " '__init_subclass__'", " '__ior__'", " '__iter__'", " '__le__'", " '__len__'", " '__lt__'", " '__module__'", " '__ne__'", " '__new__'", " '__or__'", " '__post_init__'", " '__reduce__'", " '__reduce_ex__'", " '__repr__'", " '__reversed__'", " '__ror__'", " '__setattr__'", " '__setitem__'", "

In [ ]:
# 构造新数据
new_data = {'ID':len(df)+1, '职位名称':'', '岗位描述':new_desc, '关键词':','.join(all_keywords)}

new_data
# 将预测职位转化为一个列表，避免两个列表长度不匹配导致的错误
# predict = [predict]

In [ ]:
# 找到对应的行索引
idx = df[df['职位名称'].isin(predicted_post)].index[0]

# 添加新关键词
df.loc[idx, '关键词'] += ',' + old_desc_keywords

df = data[data.index == 13] #先将指定行的数据传给df
print(df[['关键词']]) #再将指定列的数据输出
df

In [ ]:
#设置value的显示长度为1000，默认为50
pd.set_option('max_colwidth', 1000)

df